# Résolution de Sudoku avec OR-Tools

## Introduction

Google OR-Tools est une suite de logiciels d'optimisation développée par Google. Elle permet de résoudre des problèmes complexes d'optimisation combinatoire tels que la satisfaction de contraintes (CSP), la programmation linéaire (LP), la programmation linéaire mixte (MIP), et bien plus. Dans ce projet, nous nous concentrerons sur la résolution de Sudokus en utilisant différentes techniques fournies par OR-Tools.

### Types de Solveurs

1. **Solveur de Satisfaction de Contraintes (CSP)** :
   Ce solveur utilise des techniques de propagation de contraintes et de recherche pour trouver des solutions qui satisfont toutes les contraintes spécifiées. En CSP, les utilisateurs déclarent les contraintes sur les solutions faisables pour un ensemble de variables de décision. Cela permet de réduire l'espace de recherche en éliminant les valeurs non faisables. CSP se concentre sur la faisabilité (trouver une solution faisable) plutôt que sur l'optimisation (trouver une solution optimale).

2. **Solveur de Programmation Linéaire Mixte (MIP)** :
   Ce solveur combine la programmation linéaire avec des variables entières pour résoudre des problèmes d'optimisation. MIP est utilisé pour des problèmes où certaines variables doivent être entières, ce qui est utile dans des scénarios de prise de décision où les solutions doivent être discrètes. La programmation linéaire (LP) est une méthode pour obtenir le meilleur résultat (tel que le profit maximum ou le coût minimum) dans un modèle mathématique dont les exigences sont représentées par des relations linéaires.

3. **Solveur de Satisfaction de Contraintes SAT (CP-SAT)** :
   Ce solveur est basé sur un noyau SAT, utilisant des techniques avancées pour résoudre les problèmes de satisfaction de contraintes en convertissant le problème en une forme satisfiable. Le solveur SAT se concentre sur la faisabilité et utilise des techniques comme le retour en arrière chronologique et la propagation de contraintes.


Nous allons explorer ces solveurs pour résoudre des grilles de Sudoku et comparer leurs performances.

## Installation de OR-Tools

Pour utiliser OR-Tools avec C#, nous devons d'abord installer la bibliothèque. Vous pouvez suivre les instructions sur le site officiel de [Google OR-Tools](https://developers.google.com/optimization).

### Installation de OR-Tools

On appelle le package Nuget correspondant. 

In [1]:
#r "nuget: Google.OrTools"

Installed Packages Google.OrTools, 9.10.4067

## Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [2]:
#!import Sudoku-0-Environment.ipynb

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


Installed Packages XPlot.Plotly.Interactive, 4.1.0

Loading extensions from `C:\Users\Administrateur.000\.nuget\packages\xplot.plotly.interactive\4.1.0\lib\net7.0\XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger  des grilles de Sudoku et tester des solvers.

- `GetSudokus` : Renvoie des listes de Sudoku issues de fichiers de 3 difficultés différentes.
- `SolveSudoku` : effectue un test simple d'un solver sur un sudoku donné.
- `TestSolvers` : exécute les tests de performance sur plusieurs solveurs.
- `DisplayResults` : affiche les résultats des tests sous forme de graphiques.



## Implémentations des Solveurs

### Solveur par contraintes classique (CpSolver)

L'algorithme de satisfaction de contraintes (CSP) utilise les contraintes pour réduire l'espace de recherche et trouver une solution qui satisfait toutes les contraintes du problème. Nous utilisons `CpSolver` d'OR-Tools pour cette implémentation.

#### Étapes de la modélisation avec OR-Tools CSP

1. **Création du modèle** : Utiliser `CpModel()` pour créer un modèle de contraintes.
2. **Définition des variables** : Créer une grille de variables. Chaque variable représente une cellule du Sudoku, prenant une valeur entière de 1 à 9.
3. **Ajout des contraintes** : Ajouter des contraintes pour les lignes, colonnes et régions. Utiliser `AddAllDifferent()` pour déclarer que chaque ensemble de variables doit contenir des valeurs différentes.
4. **Création du solveur** : Utiliser `CpSolver()` pour créer un solveur.
5. **Résolution du modèle** : Utiliser `Solve()` pour résoudre le modèle.
6. **Extraction des résultats** : Utiliser `solver.Value()` pour obtenir les valeurs finales de toutes les variables dans la grille.


In [3]:
using Google.OrTools.ConstraintSolver;
using SimpleCPSolver = Google.OrTools.ConstraintSolver.Solver;
using SimpleConstraint = Google.OrTools.ConstraintSolver.Constraint;
using IntVar = Google.OrTools.ConstraintSolver.IntVar;
using System.Linq;
using System.Text;

public class OrToolsCPSolver : ISudokuSolver
{
    private const int GridSize = 9;
    private const int RegionSize = 3;
    public int VariableSelectionStrategy { get; set; } = SimpleCPSolver.CHOOSE_FIRST_UNBOUND;
    public int ValueSelectionStrategy { get; set; } = SimpleCPSolver.ASSIGN_MIN_VALUE;
    public SudokuGrid Solve(SudokuGrid s)
    {
        int[,] grid = s.Cells;
        SimpleCPSolver solver = new SimpleCPSolver("CpSimple");
        IntVar[,] matrix = CreateConstraints(solver, grid);
        // Parametrize DecisionBuilder
        DecisionBuilder db = solver.MakePhase(matrix.Flatten(), VariableSelectionStrategy, ValueSelectionStrategy);
        solver.NewSearch(db);
        while (solver.NextSolution())
        {
            string solvedString = BuildSolvedString(matrix);
            solver.EndSearch();
            return SudokuGrid.ReadSudoku(solvedString);
        }
        throw new Exception("Unfeasible Sudoku");
    }
    private static IntVar[,] CreateConstraints(SimpleCPSolver solver, int[,] grid)
    {
        IntVar[,] matrix = solver.MakeIntVarMatrix(GridSize, GridSize, 1, 9, "matrix");
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                if (grid[i, j] != 0)
                {
                    solver.Add(matrix[i, j] == grid[i, j]);
                }
            }
        }
        for (int i = 0; i < GridSize; i++)
        {
            solver.Add(solver.MakeAllDifferent((from j in Enumerable.Range(0, GridSize) select matrix[i, j]).ToArray()));
            solver.Add(solver.MakeAllDifferent((from j in Enumerable.Range(0, GridSize) select matrix[j, i]).ToArray()));
        }
        for (int row = 0; row < GridSize; row += RegionSize)
        {
            for (int col = 0; col < GridSize; col += RegionSize)
            {
                IntVar[] regionVars = new IntVar[RegionSize * RegionSize];
                for (int r = 0; r < RegionSize; r++)
                {
                    for (int c = 0; c < RegionSize; c++)
                    {
                        regionVars[r * RegionSize + c] = matrix[row + r, col + c];
                    }
                }
                solver.Add(solver.MakeAllDifferent(regionVars));
            }
        }
        return matrix;
    }
    private static string BuildSolvedString(IntVar[,] matrix)
    {
        StringBuilder sb = new StringBuilder();
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                sb.Append((int)matrix[i, j].Value());
            }
        }
        return sb.ToString();
    }
}


#### Test du solver CP Simple

On teste sur un sudoku de chaque difficulté

In [4]:
var solver = new OrToolsCPSolver();
var easySudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).First();
SudokuHelper.SolveSudoku(easySudoku, solver);
var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).Skip(1).First();
SudokuHelper.SolveSudoku(mediumSudoku, solver);
var hardSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Hard).First();
SudokuHelper.SolveSudoku(hardSudoku, solver);

Résolution par le solver OrToolsCPSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 29,0973 ms

Résolution par le solver OrToolsCPSolver du Sudoku:
 -------------------------------
|       5 | 3       |         | 
| 8       |         |    2    | 
|    7    |    1    | 5       | 
-------------------------------
| 4       |       5 | 3       | 
|    1    |    7    |       6 | 
|       3 | 2       |    8    | 
-------------------------------
|    6    | 5       |       9 | 
|       4 |         |    3    | 
|         |       9 | 7       | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 1  4  5 | 3  2  7 | 6  9  8 | 
| 8  3  9 | 6  5  4 | 1  2  7 | 
| 6  7  2 | 9  1  8 | 5  4  3 | 
-------------------------------
| 4  9  6 | 1  8  5 | 3  7  2 | 
| 2  1  8 | 4  7  3 | 9  5  6 | 
| 7  5  3 | 2  9  6 | 4  8  1 | 
-------------------------------
| 3  6  7 | 5  4  2 | 8  1  9 | 
| 9  8  4 | 7  6  1 | 2  3  5 | 
| 5  2  1 | 8  3  9 | 7  6  4 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 8,0663 ms

Résolution par le solver OrToolsCPSolver du Sudoku:
 -------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 4  1  7 | 3  6  9 | 8  2  5 | 
| 6  3  2 | 1  5  8 | 9  4  7 | 
| 9  5  8 | 7  2  4 | 3  1  6 | 
-------------------------------
| 8  2  5 | 4  3  7 | 1  6  9 | 
| 7  9  1 | 5  8  6 | 4  3  2 | 
| 3  4  6 | 9  1  2 | 7  5  8 | 
-------------------------------
| 2  8  9 | 6  4  3 | 5  7  1 | 
| 5  7  3 | 2  9  1 | 6  8  4 | 
| 1  6  4 | 8  7  5 | 2  9  3 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 3,7011 ms

### Solveur par contraintes SAT

Le solveur de satisfaction de contraintes SAT (CP-SAT) d'OR-Tools est un outil puissant basé sur un noyau SAT, utilisant des techniques avancées pour résoudre les problèmes de satisfaction de contraintes. Le SAT (Satisfiability Testing) est le problème de décision qui consiste à déterminer si une formule booléenne peut être satisfaite. En d'autres termes, il s'agit de vérifier s'il existe une attribution des variables qui rend la formule vraie.

#### Étapes de la modélisation avec OR-Tools SAT

1. **Création du modèle** : Utiliser `CpModel()` pour créer un modèle SAT.
2. **Définition des variables** : Créer une grille de variables où chaque cellule du Sudoku est représentée par une variable entière de 1 à 9.
3. **Ajout des contraintes** : Ajouter des contraintes pour s'assurer que chaque ligne, colonne et région contiennent des valeurs distinctes. Utiliser `AddAllDifferent()` pour garantir l'unicité des valeurs dans les sous-ensembles de la grille.
4. **Création du solveur** : Utiliser `CpSolver()` pour créer un solveur.
5. **Résolution du modèle** : Utiliser `Solve()` pour résoudre le modèle.
6. **Extraction des résultats** : Utiliser `solver.Value()` pour obtenir les valeurs finales des variables dans la grille.


In [5]:
using Google.OrTools.Sat;
using SatIntVar = Google.OrTools.Sat.IntVar;
using System;

public class OrToolsSatSolver : ISudokuSolver
{
    private const int Dimension = 9;
    private const int SubGrid = 3;
    private readonly CpSolver _solver = new CpSolver();
    
    public SudokuGrid Solve(SudokuGrid inputGrid)
    {
        (CpModel model, SatIntVar[,] grid) = CreateModel(inputGrid);
        CpSolverStatus status = _solver.Solve(model);
        if (status is CpSolverStatus.Feasible or CpSolverStatus.Optimal)
        {
            return MakeSolution(_solver, grid);
        }
        else
        {
            throw new InvalidOperationException("Sudoku grid has no solution.");
        }
    }
    private SudokuGrid MakeSolution(CpSolver solver, SatIntVar[,] grid)
    {
        SudokuGrid result = new SudokuGrid();
        for (int i = 0; i < Dimension; i++)
        {
            for (int j = 0; j < Dimension; j++)
            {
                result.Cells[i, j] = (int)solver.Value(grid[i, j]);
            }
        }
        return result;
    }
    private (CpModel model, SatIntVar[,]) CreateModel(SudokuGrid sudokuGrid)
    {
        CpModel model = new CpModel();
        SatIntVar[,] grid = new SatIntVar[Dimension, Dimension];
        CreateVariables(model, grid, sudokuGrid);
        AddConstraints(model, grid);
        return (model, grid);
    }
    private void AddConstraints(CpModel model, SatIntVar[,] grid)
    {
        for (int i = 0; i < Dimension; i++)
        {
            AddRowConstraint(model, grid, i);
            AddColumnConstraint(model, grid, i);
        }
        for (int i = 0; i < Dimension; i += SubGrid)
        {
            for (int j = 0; j < Dimension; j += SubGrid)
            {
                AddCellConstraint(model, grid, i, j);
            }
        }
    }
    private void AddCellConstraint(CpModel model, SatIntVar[,] grid, int row, int col)
    {
        SatIntVar[] cellVariables = new SatIntVar[SubGrid * SubGrid];
        for (int i = 0; i < SubGrid; i++)
        {
            for (int j = 0; j < SubGrid; j++)
            {
                cellVariables[i * SubGrid + j] = grid[row + i, col + j];
            }
        }
        model.AddAllDifferent(cellVariables);
    }
    private void AddColumnConstraint(CpModel model, SatIntVar[,] grid, int col)
    {
        SatIntVar[] colVariables = new SatIntVar[Dimension];
        for (int row = 0; row < Dimension; row++)
        {
            colVariables[row] = grid[row, col];
        }
        model.AddAllDifferent(colVariables);
    }
    private void AddRowConstraint(CpModel model, SatIntVar[,] grid, int row)
    {
        SatIntVar[] rowVariables = new SatIntVar[Dimension];
        for (int col = 0; col < Dimension; col++)
        {
            rowVariables[col] = grid[row, col];
        }
        model.AddAllDifferent(rowVariables);
    }
    private void CreateVariables(CpModel model, SatIntVar[,] grid, SudokuGrid sudokuGrid)
    {
        for (int i = 0; i < Dimension; i++)
        {
            for (int j = 0; j < Dimension; j++)
            {
                int value = sudokuGrid.Cells[i, j];
                grid[i, j] = model.NewIntVar(value == 0 ? 1 : value, value == 0 ? Dimension : value, $"Cell({i},{j})");
            }
        }
    }
}


### Solveur MIP (Mixed-Integer Programming)

L'algorithme de programmation linéaire mixte (MIP) combine la programmation linéaire et les variables entières pour résoudre des problèmes d'optimisation. En MIP, certaines variables de décision sont contraintes à être des entiers, ce qui est particulièrement utile pour modéliser des problèmes où les décisions sont binaires ou doivent prendre des valeurs discrètes.

#### Étapes de la modélisation avec OR-Tools MIP

1. **Création du modèle** : Utiliser `Solver.CreateSolver()` pour créer un solveur MIP.
2. **Définition des variables** : Créer une grille de variables. Chaque cellule du Sudoku est représentée par une variable binaire dans une matrice 3D, où chaque variable indique si une valeur spécifique (1-9) est assignée à la cellule.
3. **Ajout des contraintes** : Ajouter des contraintes pour s'assurer que chaque cellule contient exactement une valeur, et que chaque ligne, colonne et région contiennent des valeurs distinctes. Utiliser `Add()` pour ajouter des contraintes de somme et d'unicité.
4. **Résolution du modèle** : Utiliser `Solve()` pour résoudre le modèle.
5. **Extraction des résultats** : Utiliser `solution_value()` pour obtenir les valeurs finales des variables dans la grille et convertir les résultats de la représentation binaire à une matrice de valeurs entières.

La programmation linéaire mixte permet de formuler le problème de Sudoku de manière à exploiter les techniques d'optimisation linéaire et les capacités des solveurs MIP pour trouver des solutions efficaces.


In [6]:
using Google.OrTools.LinearSolver;
using LinearSolver = Google.OrTools.LinearSolver.Solver;
using LinearExpr = Google.OrTools.LinearSolver.LinearExpr;

public class OrToolsMIPSolver : ISudokuSolver
{
    private const int GridSize = 9;

    // Propriété pour choisir le type de solveur linéaire
    public string SolverID { get; set; } = "";
    public LinearSolver.OptimizationProblemType OptimizationProblemType { get; set; } = LinearSolver.OptimizationProblemType.SCIP_MIXED_INTEGER_PROGRAMMING;

    public SudokuGrid Solve(SudokuGrid s)
    {
        // Initialiser le solveur avec le type sélectionné
        LinearSolver solver;
         if (!string.IsNullOrEmpty(SolverID))
        {
             solver = LinearSolver.CreateSolver(SolverID);
        }
        else
        {
            solver = new LinearSolver("SudokuSolver", OptimizationProblemType);
        }
       
        if (solver == null)
        {
            throw new InvalidOperationException("Solver initialization failed.");
        }
        if (solver == null)
        {
            throw new InvalidOperationException("Solver initialization failed.");
        }

        Variable[,,] cells = new Variable[GridSize, GridSize, GridSize];
        InitializeVariables(s, solver, cells);

        AddConstraints(solver, cells);

        if (solver.Solve() != LinearSolver.ResultStatus.OPTIMAL)
        {
            throw new Exception("No solution found.");
        }

        return ExtractSolution(s, cells);
    }


    private void InitializeVariables(SudokuGrid s, LinearSolver solver, Variable[,,] cells)
    {
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                for (int k = 0; k < GridSize; k++)
                {
                    cells[i, j, k] = solver.MakeIntVar(0, 1, $"Cell({i},{j},{k})");
                }
                if (s.Cells[i, j] != 0)
                {
                    solver.Add(cells[i, j, s.Cells[i, j] - 1] == 1);
                }
            }
        }
    }

    private void AddConstraints(LinearSolver solver, Variable[,,] cells)
    {
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                solver.Add(Sum(solver, Enumerable.Range(0, GridSize).Select(k => cells[i, j, k])) == 1);
            }
        }

        for (int k = 0; k < GridSize; k++)
        {
            for (int i = 0; i < GridSize; i++)
            {
                solver.Add(Sum(solver, Enumerable.Range(0, GridSize).Select(j => cells[i, j, k])) == 1);
                solver.Add(Sum(solver, Enumerable.Range(0, GridSize).Select(j => cells[j, i, k])) == 1);
            }
        }

        for (int k = 0; k < GridSize; k++)
        {
            for (int i = 0; i < GridSize; i += 3)
            {
                for (int j = 0; j < GridSize; j += 3)
                {
                    solver.Add(Sum(solver, Enumerable.Range(0, 3).SelectMany(row => Enumerable.Range(0, 3).Select(col => cells[i + row, j + col, k]))) == 1);
                }
            }
        }
    }

    private LinearExpr Sum(LinearSolver solver, IEnumerable<Variable> vars)
    {
        LinearExpr sum = new();
        foreach (var v in vars)
        {
            sum += v;
        }
        return sum;
    }

    private SudokuGrid ExtractSolution(SudokuGrid s, Variable[,,] cells)
    {
        SudokuGrid solution = new SudokuGrid();
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                for (int k = 0; k < GridSize; k++)
                {
                    if (cells[i, j, k].SolutionValue() == 1)
                    {
                        solution.Cells[i, j] = k + 1;
                    }
                }
            }
        }
        
        return solution;
    }
}


## Comparaison des Performances des Solveurs

Nous allons tester nos solveurs implémentés sur des grilles de Sudoku de différentes difficultés : Facile, Moyen et Difficile. Nous mesurerons également le temps de résolution et vérifierons la validité des solutions trouvées. 

Les solveurs testés sont :
- **Solveur de Satisfaction de Contraintes (CSP) avec différents DecisionBuilder**
  - Choix par défaut
  - Choix simple
  - Choix taille minimale
- **Solveur de Programmation Linéaire Mixte (MIP)**
- **Solveur de Satisfaction de Contraintes SAT (CP-SAT)**

Le test est effectué sur un ensemble de 10 Sudokus pour chaque difficulté et chaque solveur. Les résultats incluent le nombre de Sudokus résolus et le temps moyen de résolution.



In [7]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.Linq;
using System.Threading;
using System.Threading.Tasks;
using XPlot.Plotly;
using Trace = XPlot.Plotly.Trace;
using Microsoft.DotNet.Interactive;

var cpSolverDefault = new OrToolsCPSolver();
var cpSolverSimple = new OrToolsCPSolver
{
    VariableSelectionStrategy = SimpleCPSolver.INT_VAR_SIMPLE,
    ValueSelectionStrategy = SimpleCPSolver.INT_VALUE_SIMPLE
};
var cpSolverMinSize = new OrToolsCPSolver
{
    VariableSelectionStrategy = SimpleCPSolver.CHOOSE_MIN_SIZE_LOWEST_MIN,
    ValueSelectionStrategy = SimpleCPSolver.ASSIGN_CENTER_VALUE
};

var mipSolverIDs = new[]
{
    "SCIP",
    // "GLOP",
    // "PDLP"
};

var mipSolverTypes = new[]
{
    LinearSolver.OptimizationProblemType.CLP_LINEAR_PROGRAMMING,
    LinearSolver.OptimizationProblemType.GLOP_LINEAR_PROGRAMMING,
    // LinearSolver.OptimizationProblemType.PDLP_LINEAR_PROGRAMMING,
    LinearSolver.OptimizationProblemType.SCIP_MIXED_INTEGER_PROGRAMMING,
    // LinearSolver.OptimizationProblemType.GLPK_MIXED_INTEGER_PROGRAMMING,
    LinearSolver.OptimizationProblemType.CBC_MIXED_INTEGER_PROGRAMMING,
    LinearSolver.OptimizationProblemType.BOP_INTEGER_PROGRAMMING,
    LinearSolver.OptimizationProblemType.SAT_INTEGER_PROGRAMMING,
    // LinearSolver.OptimizationProblemType.GUROBI_LINEAR_PROGRAMMING,
    // LinearSolver.OptimizationProblemType.GUROBI_MIXED_INTEGER_PROGRAMMING,
    // LinearSolver.OptimizationProblemType.CPLEX_LINEAR_PROGRAMMING,
    // LinearSolver.OptimizationProblemType.CPLEX_MIXED_INTEGER_PROGRAMMING,
    // LinearSolver.OptimizationProblemType.XPRESS_LINEAR_PROGRAMMING,
    // LinearSolver.OptimizationProblemType.XPRESS_MIXED_INTEGER_PROGRAMMING
};

var solvers = new List<(string Name, ISudokuSolver Solver)>
{
    ("CP Solver Default", cpSolverDefault),
    ("CP Solver Simple", cpSolverSimple),
    ("CP Solver Min Size", cpSolverMinSize),
    ("SAT Solver", new OrToolsSatSolver())
};

foreach (var solverID in mipSolverIDs)
{
    solvers.Add(($"MIP Solver {solverID}", new OrToolsMIPSolver { SolverID = solverID }));
}

foreach (var solverType in mipSolverTypes)
{
    solvers.Add(($"MIP Solver {solverType}", new OrToolsMIPSolver { OptimizationProblemType = solverType }));
}

// Utilisation des méthodes de benchmarking
var results = SudokuHelper.TestSolvers(solvers);
SudokuHelper.DisplayResults(results);


Testing MIP Solver SAT_INTEGER_PROGRAMMING on Hard sudokus...

<!DOCTYPE html>

<!DOCTYPE html>

<!DOCTYPE html>

### Conclusion Générale

Les résultats des tests de performance montrent une distinction claire entre les solveurs simples plus efficaces sur les problèmes simples et et les solveurs plus sophistiqués qui passent devant sur les problèmes plus difficiles.

Une observation clé de cette analyse est l'importance de la paramétrisation des solveurs. Les différents types de solveurs MIP et les stratégies de sélection de variables et de valeurs des solveurs CP peuvent considérablement influencer les performances. Par conséquent, il est crucial de sélectionner et de paramétrer les solveurs en fonction de la nature spécifique du problème à résoudre.